In [9]:
import glob
import numpy as np
import capo
# import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import imageio
import os
import hsa7458_v001 as cal
from operator import itemgetter
%matplotlib inline

In [10]:
data_dir = '/home/champ10/Desktop/Test1/'

In [11]:
data = sorted(glob.glob(''.join([data_dir, 'zen.*.npz'])))
datadict = np.load(data[0])

In [12]:
def calculate_baseline(antennae, pair):
    """
    The decimal module is necessary for keeping the number of decimal places small.
    Due to small imprecision, if more than 8 or 9 decimal places are used, 
    many baselines will be calculated that are within ~1 nanometer to ~1 picometer of each other.
    Because HERA's position precision is down to the centimeter, there is no 
    need to worry about smaller imprecision.
    """

    dx = antennae[pair[0]]['top_x'] - antennae[pair[1]]['top_x']
    dy = antennae[pair[0]]['top_y'] - antennae[pair[1]]['top_y']
    baseline = np.around([np.sqrt(dx**2. + dy**2.)],3)[0] #XXX this may need tuning
    slope = dy/np.float64(dx)
    if slope == -np.inf:
        slope = slope * -1
    elif slope == 0:
        slope = slope + 0

    ps = (pair[0],pair[1],"%.2f" % slope)
    return "%.1f" % baseline,ps

In [13]:
ex_ants=[81]
antennae = cal.prms['antpos_ideal']
baselines = {}


for antenna_i in antennae:
    if antennae[antenna_i]['top_z'] < 0.:
        continue
    if antenna_i in ex_ants:
        continue
        
    for antenna_j in antennae:
        if antennae[antenna_j]['top_z'] < 0.:
            continue
        if antenna_j in ex_ants:
            continue

        if antenna_i == antenna_j:
            continue
        elif antenna_i < antenna_j:
            pair = (antenna_i, antenna_j)
        elif antenna_i > antenna_j:
            pair = (antenna_j, antenna_i)
        
        baseline,ps = calculate_baseline(antennae, pair)
        
        if (baseline not in baselines):
            baselines[baseline] = [ps]
        elif (pair in baselines[baseline]):
            continue
        else:
            baselines[baseline].append(ps)
            
            


/home/champ10/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


In [14]:

keys = sorted(baselines) 
xr = np.arange(100.,200.,100./1024)

for i in keys:
    
    x= sorted(set(baselines[i]),key=itemgetter(2))
    print 'i=',i
    #print x
    
    seen = set()
    [item for item in x if item[2] not in seen and not seen.add(item[2])]
    seen =  sorted(seen)
    
    
    for q in seen:
        res = [k for k in x if q in k]
    
        #print res

        plt.figure(figsize=(10,10))
        for elem,antstr in enumerate(res):

            antstr1 = "%s_%s" % (res[elem][0], res[elem][1])
            #print antstr1
            
            qwerty = datadict['avgstokes_dict']
            qwerty = qwerty.item()
            
            
            stokes_i = abs(np.vectorize(complex)(qwerty['{}'.format(antstr1)]['i_real'],qwerty['{}'.format(antstr1)]['i_imag']))
            stokes_q = abs(np.vectorize(complex)(qwerty['{}'.format(antstr1)]['q_real'],qwerty['{}'.format(antstr1)]['q_imag']))
            stokes_u = abs(np.vectorize(complex)(qwerty['{}'.format(antstr1)]['u_real'],qwerty['{}'.format(antstr1)]['u_imag']))
            stokes_v = abs(np.vectorize(complex)(qwerty['{}'.format(antstr1)]['v_real'],qwerty['{}'.format(antstr1)]['v_imag']))
            
            
            plt.subplot(411)
            plt.plot(xr,stokes_i)
            plt.title('Stokes I bs:%s m:%s '%(i , q),fontsize = 10)
            plt.xlabel('Frequency (MHz)',fontsize = 8)
            plt.ylabel('Average power',fontsize = 8)
            
            
            plt.subplot(412)
            plt.plot(xr,stokes_q)
            plt.title('Stokes Q bs:%s m:%s '%(i , q),fontsize = 10)
            plt.xlabel('Frequency (MHz)',fontsize = 8)
            plt.ylabel('Average power',fontsize = 8)
            
            
            plt.subplot(413)
            plt.plot(xr,stokes_u)
            plt.title('Stokes U bs:%s m:%s '%(i , q),fontsize = 10)
            plt.xlabel('Frequency (MHz)',fontsize = 8)
            plt.ylabel('Average power',fontsize = 8)
            
            
            plt.subplot(414)
            plt.plot(xr,stokes_v)
            plt.title('Stokes V bs:%s m:%s '%(i , q),fontsize = 10)
            plt.xlabel('Frequency (MHz)',fontsize = 8)
            plt.ylabel('Average power',fontsize = 8)
            
        plt.tight_layout()
        plt.savefig('/home/champ10/Desktop/Test1/zen.2457746.avgstokes_{}_{}.png'.format(i,q))
        plt.close()

            
        

i= 14.6
i= 25.3
i= 29.2
i= 38.6
i= 43.8
i= 50.6
i= 52.6
i= 58.4
